<a href="https://colab.research.google.com/github/victorgrrtj/trabalho_final_puc/blob/main/trabalho_final_predicao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Trabalho Final - Treinamento do Modelo Preditivo

## Importação de Bibliotecas

In [1]:
!cat /etc/os-release

NAME="Ubuntu"
VERSION="20.04.5 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04.5 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal


In [2]:
%%sh
sudo su
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -

curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list > /etc/apt/sources.list.d/mssql-release.list

exit
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get install -y msodbcsql18

# optional: for bcp and sqlcmd
sudo ACCEPT_EULA=Y apt-get install -y mssql-tools18
echo 'export PATH="$PATH:/opt/mssql-tools18/bin"' >> ~/.bashrc
source ~/.bashrc
# optional: for unixODBC development headers
sudo apt-get install -y unixodbc-dev

OK


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0   7446      0 --:--:-- --:--:-- --:--:--  7446
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    89  100    89    0     0    526      0 --:--:-- --:--:-- --:--:--   526


In [3]:
%%sh
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17
# optional: for bcp and sqlcmd
sudo ACCEPT_EULA=Y apt-get install -y mssql-tools
echo 'export PATH="$PATH:/opt/mssql-tools/bin"' >> ~/.bashrc
source ~/.bashrc
# optional: for unixODBC development headers
sudo apt-get install -y unixodbc-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 https://packages.microsoft.com/ubuntu/20.04/prod focal InRelease [3,611 B]
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 https://packages.microsoft.com/ubuntu/20.04/prod focal/main amd64 Packages [200 kB]
Get:14 https://pack

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
sh: 6: source: not found


In [4]:
!sudo apt-get install unixodbc-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unixodbc-dev is already the newest version (2.3.11-1).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.


In [5]:
!pip install pyodbc
!pip install chart_studio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.5/343.5 kB 8.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.7 MB/s eta 0:00:00


In [6]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 7.4 MB/s eta 0:00:00


In [7]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.2 MB/s eta 0:00:00


In [8]:
# Importação das bibliotecas
import requests
import unidecode
import pandas as pd
import math
import time
import random
import sqlalchemy
from sqlalchemy import create_engine
import pymysql
import pyodbc
import joblib
import datetime as dt
import pytz
headers = {
    'Authorization': 'Bearer $ACCESS_TOKEN',
}

In [ ]:
user = 'adminskyblue'
pw = input('Digite a senha: ')
server = 'skyblue.database.windows.net'
database = 'skyblue'


engine = sqlalchemy.create_engine(
    "mssql+pyodbc://"+user+":"+pw+"@"+server+"/"+database+"?driver=ODBC+Driver+17+for+SQL+Server",
    fast_executemany=True
)

## Sumarização de todas as extrações.

In [10]:
""" Contém looping para todas as páginas.
Retorna um dicionário com as informações de cada imóvel.
Devido à quantidade de páginas, este processo pode demorar mais de 30 minutos."""

headers = {
    'Authorization': 'Bearer $ACCESS_TOKEN',
}

offset = 0
pag_inicial = 'https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset='+str(offset)

response = requests.get(
  pag_inicial,
  headers=headers)
json = response.json()
qtd_pag = round(json['paging']['total']/50)
qtd_itens = json['paging']['total']

print("Foram encontradas {} páginas e {} produtos.".format(qtd_pag, qtd_itens))

offset = 0
url_list = [pag_inicial]
for page in range(qtd_pag):
  offset += 50
  prox_pagina = 'https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset='+str(offset)
  url_list.append(prox_pagina)

extracted_data = []

for index, url in enumerate(url_list):
  response = requests.get(
      url,
      headers=headers)
  print("Codigo de resposta da pagina {}: {}".format(url, response.status_code))

  json = response.json()
  results = json['results']
  time.sleep(random.randint(2,6))

  for item in results:
    product_dict = {}
    product_dict['id'] = item.get('id')
    product_dict['titulo'] = item.get('title')
    product_dict['condicao'] = item.get('condition')
    product_dict['link'] = item.get('permalink')
    product_dict['preco'] = item.get('price')
    for attribute in item.get('attributes'):
      if attribute.get('name') == 'Marca':
        product_dict['marca'] = attribute.get('value_name')
      elif attribute.get('name') == 'Linha':
        product_dict['linha'] = attribute.get('value_name')
      elif attribute.get('name') == 'Modelo':
        product_dict['modelo'] = attribute.get('value_name')
    product_dict['nome_vendedor'] = item.get('seller').get('nickname')
    product_dict['nvl_vendedor'] = item.get('seller').get('seller_reputation').get('level_id')
    product_dict['vendas_concluidas'] = item.get('seller').get('seller_reputation').get('transactions').get('completed')
    product_dict['qtd_avaliacoes'] = item.get('seller').get('seller_reputation').get('transactions').get('total')
    product_dict['avlc_pos'] = item.get('seller').get('seller_reputation').get('transactions').get('ratings').get('positive')
    product_dict['avlc_neut'] = item.get('seller').get('seller_reputation').get('transactions').get('ratings').get('neutral')
    product_dict['avlc_neg'] = item.get('seller').get('seller_reputation').get('transactions').get('ratings').get('negative')
    extracted_data.append(product_dict)

  print('Finished page ' + str(index + 1))
print('Finished all pages')

Foram encontradas 17 páginas e 838 produtos.
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=0: 200
Finished page 1
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=50: 200
Finished page 2
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=100: 200
Finished page 3
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=150: 200
Finished page 4
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=200: 200
Finished page 5
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=250: 200
Finished page 6
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=300: 200
Finished page 7
Codigo de resposta da pagina https://api.mercadolibre.com/sites/MLB/search?q=Iphone%2014&offset=350: 200
Finish

In [39]:
df = pd.json_normalize(extracted_data)
df['ts_carga'] = str(dt.datetime.now(pytz.timezone('Etc/GMT+3')).strftime('%d-%m-%Y %X'))

In [40]:
df.drop(df.index[df['modelo'] == 'iPhone'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 7'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 7 Plus'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 12 Pro'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhones 8 ao 14'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 11'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 11 Pro'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 11 Pro Max'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 13'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 13 Pro'], inplace=True)
df.drop(df.index[df['modelo'] == 'iPhone 13 Pro Max'], inplace=True)
df.drop(df.index[df['titulo'].str.contains('iphone', case=False) == False], inplace=True)
df.drop(df.index[df['titulo'].str.contains('14', case=False) == False], inplace=True)
df.reset_index(inplace=True, drop=True)

In [41]:
df['titulo'] = df['titulo'].apply(lambda x: unidecode.unidecode(x))

In [42]:
model = joblib.load("model.joblib")
cv = joblib.load("cv.joblib")

In [43]:
modelos_features = cv.transform(df['titulo'].values)

df['modelo'] = model.predict(modelos_features)

In [44]:
df.to_sql("vendas", engine, index=False, chunksize=500, if_exists = "append")

-2